In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install langchain_community neo4j langchain langchain_groq

### Setup Neo4j Database

In [4]:
from langchain_community.graphs import Neo4jGraph

NEO4J_URL = "neo4j+s://demo.neo4jlabs.com"
NEO4J_DATABASE = "recommendations"
NEO4J_USERNAME = "recommendations"
NEO4J_PASSWORD = "recommendations"

graph = Neo4jGraph(
    url=NEO4J_URL,
    database=NEO4J_DATABASE,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    sanitize=True
)
print(graph.schema)

Node properties:
Movie {posterEmbedding: LIST, url: STRING, runtime: INTEGER, revenue: INTEGER, budget: INTEGER, plotEmbedding: LIST, imdbRating: FLOAT, released: STRING, countries: LIST, languages: LIST, plot: STRING, imdbVotes: INTEGER, imdbId: STRING, year: INTEGER, poster: STRING, movieId: STRING, tmdbId: STRING, title: STRING}
Genre {name: STRING}
User {userId: STRING, name: STRING}
Actor {url: STRING, bornIn: STRING, bio: STRING, died: DATE, born: DATE, imdbId: STRING, name: STRING, poster: STRING, tmdbId: STRING}
Director {url: STRING, bornIn: STRING, born: DATE, died: DATE, tmdbId: STRING, imdbId: STRING, name: STRING, poster: STRING, bio: STRING}
Person {url: STRING, died: DATE, bornIn: STRING, born: DATE, imdbId: STRING, name: STRING, poster: STRING, tmdbId: STRING, bio: STRING}
Relationship properties:
RATED {rating: FLOAT, timestamp: INTEGER}
ACTED_IN {role: STRING}
DIRECTED {role: STRING}
The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:User)-[:RATED]->(:Movie)
(:Actor)

New Update from Langchain (09/05/24): an enhanced schema parameter representation that samples the database values and return them to the LLM to be able to generate more accurate Cypher statements

https://python.langchain.com/v0.1/docs/integrations/graphs/neo4j_cypher/#enhanced-schema-information

In [5]:
graph = Neo4jGraph(
    url=NEO4J_URL,
    database=NEO4J_DATABASE,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    sanitize=True,
    enhanced_schema=True
)
print(graph.schema)

Node properties:
- **Movie**
  - `url`: STRING Example: "https://themoviedb.org/movie/862"
  - `runtime`: INTEGER Min: 2, Max: 910
  - `revenue`: INTEGER Min: 1, Max: 2787965087
  - `imdbRating`: FLOAT Min: 1.6, Max: 9.6
  - `released`: STRING Example: "1995-11-22"
  - `countries`: LIST Min Size: 1, Max Size: 16
  - `languages`: LIST Min Size: 1, Max Size: 19
  - `plot`: STRING Example: "A cowboy doll is profoundly threatened and jealous"
  - `imdbVotes`: INTEGER Min: 13, Max: 1626900
  - `imdbId`: STRING Example: "0114709"
  - `year`: INTEGER Min: 1902, Max: 2016
  - `poster`: STRING Example: "https://image.tmdb.org/t/p/w440_and_h660_face/uXDf"
  - `movieId`: STRING Example: "1"
  - `tmdbId`: STRING Example: "862"
  - `title`: STRING Example: "Toy Story"
  - `budget`: INTEGER Min: 1, Max: 380000000
- **Genre**
  - `name`: STRING Example: "Adventure"
- **User**
  - `userId`: STRING Example: "1"
  - `name`: STRING Example: "Omar Huffman"
- **Actor**
  - `url`: STRING Example: "https://t

### Pre-finetune Test

In [6]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain.chains import GraphCypherQAChain

GROQ_API_KEY = userdata.get("GROQ_API_KEY")
GROQ_MODEL = "llama3-8b-8192"

model = ChatGroq(
    temperature=0.0,
    model=GROQ_MODEL,
    groq_api_key=GROQ_API_KEY,
)

chain = GraphCypherQAChain.from_llm(graph=graph, llm=model, verbose=True)

In [7]:
questions = [
    "Who is the oldest director?",
    "Find all directors who have directed a movie in Spanish language.",
    "Give me 5 movies where a director has also acted?",
    "List all movies with an IMDb rating greater than 5 that have been directed by a director born in China."
]

# POSSIBLE CORRECT CYPHER QUERY
# 1. MATCH (d:Director) WHERE d.born IS NOT NULL RETURN d ORDER BY d.born ASC LIMIT 1
# 2. MATCH (d:Director)-[:DIRECTED]->(m:Movie) WHERE 'Spanish' IN m.languages RETURN d.name
# 3. MATCH (d:Director)-[:ACTED_IN]->(m:Movie) WHERE exists{ (d)-[:DIRECTED]->(m) } RETURN m.title AS MovieTitle, m.movieId AS MovieID LIMIT 5
# 4. MATCH (m:Movie)<-[:DIRECTED]-(d:Director) WHERE m.imdbRating > 5 AND d.bornIn = 'China' RETURN m

for q in questions:
    print("\n", q)
    try:
        result = chain.invoke(q)['result']
        print(result)
    except:
        pass


 Who is the oldest director?


> Entering new GraphCypherQAChain chain...


Generated Cypher:
MATCH (d:Director)-[:born]->(b) RETURN d.name AS director, b.born AS birth_date ORDER BY b.born ASC LIMIT 1;
Full Context:
[]

> Finished chain.
I don't know the answer.

 Find all directors who have directed a movie in Spanish language.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Director)-[:DIRECTED]->(m:Movie)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(lang:Genre {name: "Spanish"}) RETURN d;
Full Context:
[]

> Finished chain.
I don't know the answer.

 Give me 5 movies where a director has also acted?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Director)-[:ACTED_IN]->(m:Movie) RETURN m LIMIT 5;
Full Context:
[{'m': {'languages': ['English'], 'year': 1919, 'imdbId': '0009932', 'runtime': 12, 'imdbRating': 6.1, 'movieId': '72626', 'countries': ['USA'], 'imdbVotes': 503, 'title': 'Billy Blazes, Esq.', 'url': 'https://themoviedb.org/movie/53516', 'tmdbId': '53516', 'plot': 'Billy Blazes confronts Crooked Charley, who h

New Update from Langchain: use validate_cypher parameter with enhanced schema parameter to get the best results.

> `validate_cypher = True`

In [9]:
chain = GraphCypherQAChain.from_llm(graph=graph, llm=model, verbose=True, validate_cypher=True)

for q in questions:
    print("\n", q)
    try:
        result = chain.invoke(q)['result']
        print(result)
    except:
        pass


 Who is the oldest director?


> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.
I don't know the answer.

 Find all directors who have directed a movie in Spanish language.


> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.
I don't know the answer.

 Give me 5 movies where a director has also acted?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Director)-[:ACTED_IN]->(m:Movie) RETURN m LIMIT 5;
Full Context:
[{'m': {'languages': ['English'], 'year': 1919, 'imdbId': '0009932', 'runtime': 12, 'imdbRating': 6.1, 'movieId': '72626', 'countries': ['USA'], 'imdbVotes': 503, 'title': 'Billy Blazes, Esq.', 'url': 'https://themoviedb.org/movie/53516', 'tmdbId': '53516', 'plot': 'Billy Blazes confronts Crooked Charley, who has been ruling the town of Peaceful Vale through fear and violence.', 'released': '1919-07-06'}}, {'m': {'languages': ['English'], 'year': 1925, 'imdbI

### Create Dataset


In [10]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = f"Convert text to cypher query based on this schema: {graph.schema}"
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instructions, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [11]:
!wget https://raw.githubusercontent.com/martin-fabbri/graph-llm-agents/main/notebooks/data/text_to_cypher_dataset.csv

--2024-06-27 07:21:38--  https://raw.githubusercontent.com/martin-fabbri/graph-llm-agents/main/notebooks/data/text_to_cypher_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2426996 (2.3M) [text/plain]
Saving to: ‘text_to_cypher_dataset.csv’

text_to_cypher_data 100%[===================>]   2.31M  --.-KB/s    in 0.07s   

2024-06-27 07:21:39 (31.4 MB/s) - ‘text_to_cypher_dataset.csv’ saved [2426996/2426996]



In [12]:
import pandas as pd

df = pd.read_csv("text_to_cypher_dataset.csv")
df = df[(df["database"] == "recommendations") & (df["syntax_error"] == False) & (df["timeout"] == False)]
df

,question,cypher,type,database,syntax_error,timeout,returns_results,false_schema
7275,What are the top 5 movies with a runtime great...,MATCH (m:Movie)\nWHERE m.runtime > 120\nRETURN...,Simple Retrieval Queries,recommendations,False,False,True,NaN
7276,List the first 3 genres with movies having an ...,MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)\nWHERE ...,Verbose query,recommendations,False,False,True,NaN
7277,List the first 5 directors who have a biograph...,MATCH (d:Director)\nWHERE d.bio IS NOT NULL\nR...,Simple Retrieval Queries,recommendations,False,False,True,NaN
7278,Which 3 movies have the most detailed plot des...,"MATCH (m:Movie)\nRETURN m.title, m.plot\nORDER...",Simple Retrieval Queries,recommendations,False,False,True,NaN
7279,Show the top 5 actors who have acted in movies...,MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)<-[:DIRE...,Simple Retrieval Queries,recommendations,False,False,True,NaN
...,...,...,...,...,...,...,...,...
8067,Which movies have been acted in by more than 1...,MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)\nWITH m...,Complex Retrieval Queries,recommendations,False,False,True,NaN
8068,Find all movies where the director has directe...,MATCH (d:Director)-[:DIRECTED]->(m:Movie)\nWIT...,Complex Retrieval Queries,recommendations,False,False,False,NaN
8069,Find all movies that have a plot mentioning 'h...,MATCH (m:Movie)\nWHERE m.plot CONTAINS 'hero'\...,Complex Retrieval Queries,recommendations,False,False,True,NaN
8070,Which movies have been rated the highest by us...,"MATCH (u:User)-[r:RATED]->(m:Movie)\nWITH u, c...",Complex Retrieval Queries,recommendations,False,False,True,NaN


In [13]:
df = df[["question","cypher"]]
df.rename(columns={"question": "input", "cypher": "output"}, inplace=True)
df.reset_index(drop=True, inplace=True)
df

<ipython-input-13-87097192af22>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"question": "input", "cypher": "output"}, inplace=True)


,input,output
0,What are the top 5 movies with a runtime great...,MATCH (m:Movie)\nWHERE m.runtime > 120\nRETURN...
1,List the first 3 genres with movies having an ...,MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)\nWHERE ...
2,List the first 5 directors who have a biograph...,MATCH (d:Director)\nWHERE d.bio IS NOT NULL\nR...
3,Which 3 movies have the most detailed plot des...,"MATCH (m:Movie)\nRETURN m.title, m.plot\nORDER..."
4,Show the top 5 actors who have acted in movies...,MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)<-[:DIRE...
...,...,...
757,Which movies have been acted in by more than 1...,MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)\nWITH m...
758,Find all movies where the director has directe...,MATCH (d:Director)-[:DIRECTED]->(m:Movie)\nWIT...
759,Find all movies that have a plot mentioning 'h...,MATCH (m:Movie)\nWHERE m.plot CONTAINS 'hero'\...
760,Which movies have been rated the highest by us...,"MATCH (u:User)-[r:RATED]->(m:Movie)\nWITH u, c..."


In [14]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)
dataset = dataset.map(formatting_prompts_func, batched = True)
dataset

Parameter 'function'=<function formatting_prompts_func at 0x7cdaa8d07eb0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/762 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'text'],
    num_rows: 762
})

In [15]:
dataset[0]

{'input': 'What are the top 5 movies with a runtime greater than 120 minutes?',
 'output': 'MATCH (m:Movie)\nWHERE m.runtime > 120\nRETURN m\nORDER BY m.runtime DESC\nLIMIT 5',
 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nConvert text to cypher query based on this schema: Node properties:\n- **Movie**\n  - `url`: STRING Example: "https://themoviedb.org/movie/862"\n  - `runtime`: INTEGER Min: 2, Max: 910\n  - `revenue`: INTEGER Min: 1, Max: 2787965087\n  - `imdbRating`: FLOAT Min: 1.6, Max: 9.6\n  - `released`: STRING Example: "1995-11-22"\n  - `countries`: LIST Min Size: 1, Max Size: 16\n  - `languages`: LIST Min Size: 1, Max Size: 19\n  - `plot`: STRING Example: "A cowboy doll is profoundly threatened and jealous"\n  - `imdbVotes`: INTEGER Min: 13, Max: 1626900\n  - `imdbId`: STRING Example: "0114709"\n  - `year`: INTEGER Min: 1902, Max: 2016\

### Train the model

Now let's use Huggingface TRL's SFTTrainer! More docs here: TRL SFT docs. We do 60 steps to speed things up, but you can set num_train_epochs=1 for a full run, and turn off max_steps=None. We also support TRL's DPOTrainer!

In [17]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [18]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [19]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/762 [00:00<?, ? examples/s]

In [21]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
10.912 GB of memory reserved.


In [22]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 762 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 285
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.138300
2,1.132200
3,1.134800
4,1.105900
5,1.027300
6,0.914100
7,0.791300
8,0.696400
9,0.592000
10,0.470100


In [23]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

942.859 seconds used for training.
15.71 minutes used for training.
Peak reserved memory = 10.912 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 27.581 %.
Peak reserved memory for training % of max memory = 0.0 %.


### Inference

In [24]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        f"Convert text to cypher query based on this schema: {graph.schema}", # instruction
        "Who is the oldest director?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Convert text to cypher query based on this schema: Node properties:
- **Movie**
  - `url`: STRING Example: "https://themoviedb.org/movie/862"
  - `runtime`: INTEGER Min: 2, Max: 910
  - `revenue`: INTEGER Min: 1, Max: 2787965087
  - `imdbRating`: FLOAT Min: 1.6, Max: 9.6
  - `released`: STRING Example: "1995-11-22"
  - `countries`: LIST Min Size: 1, Max Size: 16
  - `languages`: LIST Min Size: 1, Max Size: 19
  - `plot`: STRING Example: "A cowboy doll is profoundly threatened and jealous"
  - `imdbVotes`: INTEGER Min: 13, Max: 1626900
  - `imdbId`: STRING Example: "0114709"
  - `year`: INTEGER Min: 1902, Max: 2016
  - `poster`: STRING Example: "https://image.tmdb.org/t/p/w440_and_h660_face/uXDf"
  - `movieId`: STRING Example: "1"
  - `tmdbId`: STRING Example: "862"
  - `title`: STRING Example: "T


### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [25]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [26]:
from google.colab import userdata
HF_API_KEY = userdata.get("HF_API_KEY")

model.push_to_hub("martinfabbri/llama3_text_to_cypher", token = HF_API_KEY)
tokenizer.push_to_hub("martinfabbri/llama3_text_to_cypher", token = HF_API_KEY)

README.md:   0%|          | 0.00/579 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/martinfabbri/llama3_text_to_cypher


In [27]:
model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
model.push_to_hub_gguf("martinfabbri/llama3_text_to_cypher", tokenizer, quantization_method = "f16", token = HF_API_KEY)

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 60.82 out of 83.48 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 46.24it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GUUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at model into f16 GGUF format.
The output location will be ./model/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 8192
INFO:hf-to-gguf:gguf: embedding length = 4096
INFO:hf-to-gguf:gguf: feed forward length = 14336
INFO:hf-to-gguf:gguf: head count = 32
INFO:hf-to-gguf:gguf: key-value head count = 8
INFO:hf-to-gguf:gguf: rope theta = 500000.0
INFO:hf-to-gguf:gguf: rms norm epsilon = 1e

100%|██████████| 32/32 [00:00<00:00, 69.33it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GUUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at martinfabbri/llama3_text_to_cypher into f16 GGUF format.
The output location will be ./martinfabbri/llama3_text_to_cypher/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: llama3_text_to_cypher
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 8192
INFO:hf-to-gguf:gguf: embedding length = 4096
INFO:hf-to-gguf:gguf: feed forward lengt

unsloth.F16.gguf:   0%|          | 0.00/16.1G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/martinfabbri/llama3_text_to_cypher


### Evaluating

In [33]:
inputs = tokenizer(
[
    prompt.format(
        f"Convert text to cypher query based on this schema: {graph.schema}", # instruction
        "Who is the oldest director?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
result = tokenizer.batch_decode(outputs)
response = result[0].split("### Response:")[1].split("###")[0].strip().replace("<|end_of_text|>", "")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [34]:
print(response)

MATCH (d:Director)
RETURN d.name, d.born
ORDER BY d.born ASC
LIMIT 1


In [35]:
context = graph.query(response)
context

[{'d.name': 'Georges Méliès', 'd.born': neo4j.time.Date(1861, 12, 8)}]

In [36]:
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from google.colab import userdata

groq_api_key = userdata.get('GROQ_API_KEY')

CYPHER_QA_TEMPLATE = """You convert context to a final answer. Understand the question, the context, then generate result.
Here is an example:

Question: Who is the director of Harry Potter 1 and 8?
Context: [{{d.name: Chris Columbus, d.born: 10 September 1958}},{{d.name: David Yates, d.born: 8 October 1963}}]
Helpful Answer: Chris Columbus and David Yates is the director of Harry Potter

Follow this example when generating answers.
Answer in short, don't hallucinate!
Question: {question}
Information: {context}
Helpful Answer:
"""

qa_prompt = ChatPromptTemplate.from_template(CYPHER_QA_TEMPLATE)
output_parser = StrOutputParser()
llm = ChatGroq(temperature=0, model_name="llama3-8b-8192", groq_api_key = groq_api_key)
chain = qa_prompt | llm | output_parser

context = graph.query(response)
question = 'Who is the oldest director?'

chain.invoke({"context":context , "question":question})


'Georges Méliès'

In [37]:
questions = ["Who is the oldest director?",
             "Find all directors who have directed a movie in Spanish language.",
             "Give me 5 movies where a director has also acted?",
             "List all movies with an IMDb rating greater than 5 that have been directed by a director born in China."
             ]

def generate_cypher_query(question):
  inputs = tokenizer(
  [
      prompt.format(
          f"Convert text to cypher query based on this schema: {graph.schema}", # instruction
          question, # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
  result = tokenizer.batch_decode(outputs)
  cypher_query = result[0].split("### Response:")[1].split("###")[0].strip().replace("<|end_of_text|>", "")
  return cypher_query

for q in questions:
    print("\n",q)
    cypher_query = generate_cypher_query(q)
    print(cypher_query)
    context = graph.query(cypher_query)
    print('context: ', context)
    result = chain.invoke({"context":context , "question":q})
    print(result)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



 Who is the oldest director?
MATCH (d:Director)
WHERE d.born IS NOT NULL
RETURN d
ORDER BY d.born ASC
LIMIT 1
context:  [{'d': {'bornIn': 'Paris, France', 'tmdbId': '11523', 'imdbId': '0617588', 'born': neo4j.time.Date(1861, 12, 8), 'name': 'Georges Méliès', 'bio': 'Georges Méliès, full name Marie-Georges-Jean Méliès, was a French illusionist and filmmaker famous for leading many technical and narrative developments in the earliest days of cinema.  One of the first filmmakers to use multiple exposures, time-lapse photography, tracking shots, dissolves, and hand-painted color in his work, Méliès pioneered effects that would define cinematic special effects for decades to come...', 'died': neo4j.time.Date(1938, 1, 21), 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/ba3Kfc01Dbigt41lyuFoZR7gmv1.jpg', 'url': 'https://themoviedb.org/person/11523'}}]


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Georges Méliès

 Find all directors who have directed a movie in Spanish language.
MATCH (d:Director)-[:DIRECTED]->(m:Movie)
WHERE 'Spanish' IN m.languages
RETURN d.name, collect(m.title) AS movies
context:  [{'d.name': 'Alejandro Jodorowsky', 'movies': ['Topo, El', 'Fando and Lis (Fando y Lis)']}, {'d.name': 'Alfonso Arau', 'movies': ['Like Water for Chocolate (Como agua para chocolate)']}, {'d.name': 'Abel Ferrara', 'movies': ['King of New York']}, {'d.name': 'Nacho Vigalondo', 'movies': ['Timecrimes (Cronocrímenes, Los)']}, {'d.name': 'Luis Buñuel', 'movies': ['Tristana', 'Nazarin (Nazarín)', 'Simon of the Desert (Simón del desierto)', 'Viridiana', 'Exterminating Angel, The (Ángel exterminador, El)']}, {'d.name': 'Mikhail Kalatozov', 'movies': ['I Am Cuba (Soy Cuba/Ya Kuba)']}, {'d.name': 'Les Blank', 'movies': ['Burden of Dreams']}, {'d.name': 'Juan Piquer Simón', 'movies': ['Pieces (Mil gritos tiene la noche) (One Thousand Cries Has the Night)']}, {'d.name': 'Pedro Almodóvar', 'mo

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here is the answer:

The directors who have directed a movie in Spanish language are:

* Alejandro Jodorowsky
* Alfonso Arau
* Abel Ferrara
* Nacho Vigalondo
* Luis Buñuel
* Mikhail Kalatozov
* Les Blank
* Juan Piquer Simón
* Pedro Almodóvar
* Gregory Nava
* Luis Puenzo
* Barbet Schroeder
* Martin Campbell
* Steven Soderbergh
* Luis Mandoki
* Fernando Trueba
* Guillermo del Toro
* Fernando E. Solanas
* Tomás Gutiérrez Alea
* Jorge Fons
* Álex de la Iglesia
* Alfonso Cuarón
* Alejandro Amenábar
* Juan José Campanella
* Julio Medem
* Walter Salles
* José Luis Cuerda
* Juan Carlos Fresnadillo
* Fabián Bielinsky
* Juan Pablo Rebella
* Agustín Díaz Yanes
* Alejandro Agresti
* Fernando León de Aranoa
* Alejandro González Iñárritu
* Dunia Ayaso
* Félix Sabroso
* Sebastián Cordero
* Joshua Marston
* Daniel Sánchez Arévalo
* Damián Szifrón
* J.A. Bayona
* Patricia Riggen
* Isidro Ortiz
* Luis Piedrahita
* Rodrigo Sopeña
* Jaume Balagueró
* Christian Molina
* Cary Joji Fukunaga
* Gustavo Taretto

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on the provided information, here are 5 movies where a director has also acted:

1. **Safety Last! (1923)** - Director: Fred Newmeyer, Actor: Harold Lloyd
2. **The Freshman (1925)** - Director: Sam Taylor, Actor: Harold Lloyd
3. **The Kid Brother (1927)** - Director: Sidney Franklin, Actor: Bobby Buntrock
4. **The Golem (1920)** - Director: Paul Wegener, Actor: Paul Wegener
5. **Billy Blazes, Esq. (1919)** - Director: Edward Warren, Actor: Edward Warren

 List all movies with an IMDb rating greater than 5 that have been directed by a director born in China.
MATCH (d:Director)-[:DIRECTED]->(m:Movie)
WHERE d.bornIn = 'China' AND m.imdbRating > 5
RETURN m
context:  [{'m': {'languages': ['Cantonese', ' Mandarin'], 'year': 1991, 'imdbId': '0102293', 'runtime': 91, 'imdbRating': 7.1, 'movieId': '26736', 'countries': ['Hong Kong', ' Japan'], 'imdbVotes': 9590, 'title': 'Riki-Oh: The Story of Ricky (Lik Wong)', 'url': 'https://themoviedb.org/movie/17467', 'tmdbId': '17467', 'plot': 'A yo